In [4]:
import pandas as pd
from api.core.mongo import db
from langchain_text_splitters import RecursiveJsonSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_mongodb import MongoDBAtlasVectorSearch

In [13]:
rows = pd.DataFrame(list(db["products"].find({}))).drop(columns=["_id"]).to_dict(orient="records")

In [14]:
splitter = RecursiveJsonSplitter(max_chunk_size=100)
docs = splitter.create_documents(texts=rows[0:1000])

In [7]:
# Load the embedding model (https://huggingface.co/nomic-ai/nomic-embed-text-v1")
model = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1", model_kwargs={ "trust_remote_code": True })

C:\Users\holmt\PycharmProjects\sunhacks-2024\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
C:\Users\holmt\PycharmProjects\sunhacks-2024\venv\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\holmt\.cache\huggingface\hub\models--nomic-ai--nomic-embed-text-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer

In [16]:
# Store the data as vector embeddings in Atlas

#: import os
#: os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

vector_store = MongoDBAtlasVectorSearch.from_documents(
    documents = docs,
    embedding = model,
    collection = db["embeddings"],
    index_name = "vector_index"
)